In [1]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# data
data = pd.read_csv("data/songs_with_genre.csv", index_col=0, header=[0, 1, 2])
data.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        05        06   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.012226  0.012111   
3           1.366848  1.054094  0.108103  0.619185  ...  0.014212  0.017740   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.012691  0.014759   
10          1.770694  1.604566  0.521217  1.982386  ...  0.017952  0.013921   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.016322  0.015819   

feature           zcr                                                         \
statistics   kurtosis       max      mean    median  min      skew       std   
number             01        01        01        01   01        01        01   
track_id                                                                       
2            5.758890  0.459473  0.085629  0.071289  0.0  2.089872  0.061448   
3            2.824694  0.466309  0.084578  0.063965  0.0  1.716724  0.069330   
5            6.808415  0.375000  0.053114  0.041504  0.0  2.193303  0.044861   
10          21.434212  0.452148  0.077515  0.071777  0.0  3.542325  0.040800   
134          4.731087  0.419434  0.064370  0.050781  0.0  1.806106  0.054623   

feature                genre_id  
statistics Unnamed: 519_level_1  
number     Unnamed: 519_level_2  
track_id                         
2                            20  
3                            20  
5                            20  
10                            9  
134                          20  

[5 rows x 519 columns]

In [3]:
# targets
generes = data["genre_id"]
data.drop(columns=["genre_id"], inplace=True)


C:\Users\talia\AppData\Local\Temp\ipykernel_12244\1480151535.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  data.drop(columns=["genre_id"], inplace=True)


In [4]:
generes.columns = range(generes.shape[1])
# generes

In [5]:
inx = pd.read_csv("data/genres_index.csv", index_col=0)
# inx.head()

In [6]:
vocabulary_size = 518
categories = 161  # 161
x = tf.placeholder(tf.float32, [None, vocabulary_size])
y_ = tf.placeholder(tf.float32, [None, categories])
W = tf.Variable(tf.zeros([vocabulary_size, categories]))
b = tf.Variable(tf.zeros([categories]))

z = tf.matmul(x, W) + b
pred = tf.nn.softmax(z)
# loss = -tf.reduce_mean(y_ * tf.log(pred))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(y_, z))

update = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [7]:
data_x = data
targets = OneHotEncoder().fit_transform(inx).toarray()
# print(generes)
# print(data_x)

In [8]:
generes[0] = generes[0] - 1
data_y = [targets[i - 1] for i in generes[0]]
# max(generes[0])

C:\Users\talia\AppData\Local\Temp\ipykernel_12244\3020414886.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generes[0] = generes[0] - 1


In [9]:
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(data_x, data_y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data).
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

print(X_train.shape), print(len(y_train))
print(X_valid.shape), print(len(y_valid))
print(X_test.shape), print(len(y_test))

(39678, 518)
39678
(4960, 518)
4960
(4960, 518)
4960


(None, None)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [11]:
# test:
def calc_acc_train():
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = sess.run(accuracy, feed_dict={x: X_train, y_: y_train})
    print("Accuracy:", acc * 100)
    return acc

def calc_acc_valid():
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = sess.run(accuracy, feed_dict={x: X_valid, y_: y_valid})
    print("Validation Accuracy: ", acc * 100, "%")
    return acc

def calc_acc_test():
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    acc = sess.run(accuracy, feed_dict={x: X_test, y_: y_test})
    print("Test Accuracy:", acc * 100, "%")
    return acc

In [12]:
max_acc = 0
itr = 0
for i in range(0, 20001):
    sess.run(update, feed_dict={x: X_train, y_: y_train})
    if i % 1000 == 0:
        print('Iteration:', i, '\n',
              ' W:', sess.run(W), '\n',
              ' b:', sess.run(b), '\n',
              'loss:', loss.eval(session=sess, feed_dict={x: X_train, y_: y_train}))
        acc = calc_acc_train()
        if max_acc < acc:
            max_acc = acc
            saver.save(sess, "mySoftmax", global_step=1)
            itr = i

        # saver.restore(sess, "sm_data.txt")


Iteration: 0 
  W: [[-9.9974772e-05 -9.9984572e-05 -9.9997800e-05 ... -9.9976962e-05
  -9.9976962e-05  9.9995923e-05]
 [-9.9958925e-05 -9.9987163e-05 -9.9997756e-05 ... -9.9965422e-05
  -9.9965422e-05 -9.9994715e-05]
 [-9.9976991e-05 -9.9987854e-05 -9.9997975e-05 ... -9.9982753e-05
  -9.9982753e-05  9.9995494e-05]
 ...
 [-9.7390875e-05  9.0773217e-05  9.8321019e-05 ... -9.8118668e-05
  -9.8118668e-05  9.9857061e-05]
 [-9.9997153e-05  9.9998520e-05  9.9999466e-05 ... -9.9998309e-05
  -9.9998309e-05  9.9999867e-05]
 [-9.9814381e-05  9.9794095e-05  9.9892983e-05 ... -9.9873854e-05
  -9.9873854e-05  9.9985460e-05]] 
  b: [-9.9992132e-05  9.9994104e-05  9.9998237e-05 -9.9994919e-05
 -9.9994919e-05  9.9993682e-05 -9.9987265e-05  9.9999233e-05
 -9.9994919e-05  9.9999881e-05 -9.9994446e-05 -9.9988116e-05
  9.9999830e-05 -9.9994919e-05  9.9999379e-05 -9.9994919e-05
 -9.9994919e-05  9.9986450e-05  9.9999525e-05 -9.9994919e-05
 -9.9994919e-05 -9.9994919e-05 -9.9994919e-05 -9.9994919e-05
 -9.99949

In [13]:
print(max_acc)
print(itr)

0.659131
20000


In [17]:
valid_acc = calc_acc_valid()


Validation Accuracy:  64.21371102333069 %


In [18]:
test_acc = calc_acc_test()

Test Accuracy: 63.62903118133545 %
